In [1]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

import pandas as pd
import numpy as np

from create_staging_tables import create_staging_merged_initial_df

In [2]:
engine = create_engine('postgresql://localhost:5432/wa_leg_staging')

In [3]:
con = engine.connect()

In [4]:
staging_vote_df = pd.read_sql_query('select * from "vote"',con=engine)

In [5]:
staging_bill_df = pd.read_sql_query('select * from "bill"',con=engine)

In [ ]:
def create_staging_merged_initial_df(staging_vote_df, staging_bill_df):
    '''Create merged_initial by merging staging_bill_df and staging_vote_df on the bill_unique field.
    Input
    staging_bill_df: pandas dataframe loaded from wa_leg_staging database, bill table
    staging_vote_df:pandas dataframe loaded from wa_leg_staging database, vote table
    '''
    staging_bill_df['bill_unique'] = staging_bill_df['biennium'] + ' ' + staging_bill_df['bill_id']
    return staging_vote_df.merge(staging_bill_df, how='left', on='bill_unique')

In [ ]:
MERGED_initial = create_staging_merged_initial_df(staging_vote_df, staging_bill_df)

In [ ]:
MERGED_initial.head()

In [ ]:
len(MERGED_initial)

In [ ]:
MERGED_initial[0:1].to_sql('merged_initial', con, if_exists='replace', index=False)

In [ ]:
staging_vote_df.head()

In [ ]:
staging_bill_df['htm_create_date'] = pd.to_datetime(staging_bill_df['htm_create_date'])

In [ ]:
staging_bill_df['htm_last_modified_date'] = pd.to_datetime(staging_bill_df['htm_last_modified_date'])

In [ ]:
staging_bill_df[staging_bill_df['bill_unique'] == '1993-94 HB 1790']

In [ ]:
150*19000

In [ ]:
staging_bill_df['unique_id'] = np.linspace(1, len(staging_bill_df), len(staging_bill_df))

In [ ]:
unique_vote_dates = staging_vote_df[['bill_unique', 'vote_date']]

In [ ]:
unique_vote_dates.head()

In [ ]:
unique_vote_dates.drop_duplicates(keep='first', inplace=True)

In [ ]:
unique_vote_dates['unique_id'] = np.nan

In [ ]:
unique_vote_dates = unique_vote_dates.reset_index()

In [ ]:
unique_vote_dates.head()

In [13]:
def create_staging_unique_vote_dates_df(staging_bill_df, staging_vote_df):
    '''Identify the exact bill that the legislators voted on. Create a dataframe that conists of all vote_date 
    and bill_unique pairs. Create a null unique_id field that labels each bill with a unique ID, and begins as 
    null for unique_vote_dates. For each row in unique_vote_dates identify the bill that was created closest to, 
    but before, the vote date. Set the unique_id of that bill to the unique_id of that vote. This will be use to 
    later join the bill_df to the vote_df.'''
    
    staging_bill_df['htm_create_date'] =        pd.to_datetime(staging_bill_df['htm_create_date'])
    staging_bill_df['htm_last_modified_date'] = pd.to_datetime(staging_bill_df['htm_last_modified_date'])
    
    unique_vote_dates = staging_vote_df[['bill_unique', 'vote_date']]
    unique_vote_dates.drop_duplicates(keep='first', inplace=True)
    unique_vote_dates = unique_vote_dates.reset_index()
    
    unique_vote_dates['unique_id'] = np.nan
    staging_bill_df['unique_id'] = np.linspace(1, len(staging_bill_df), len(staging_bill_df))
    

    for i1, row in unique_vote_dates.iterrows():
        time_diffs = {}
        bill_options = staging_bill_df[staging_bill_df['bill_unique'] == row['bill_unique']]
        
        for i2, option in bill_options.iterrows():
            time_diff = option['htm_create_date'] - row['vote_date']
            
            if time_diff <= pd.to_timedelta('0'):
                time_diffs[time_diff] = option['unique_id']
            if time_diff > pd.to_timedelta('0'):
                time_diff -= pd.to_timedelta('-1000 days')
                time_diffs[time_diff] = option['unique_id']
                
        if len(time_diffs) > 0:
            bill_voted_on = time_diffs[max(time_diffs)]
            unique_vote_dates.iloc[i1, -1] = bill_voted_on
            
    return unique_vote_dates

In [ ]:
unique_vote_dates.to_sql('unique_vote_dates', con, if_exists='replace', index=False)

In [ ]:
len(x)